In [1]:
import pandas as pd
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

In [2]:
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [4]:
df[df['document']=='5170565b']

,answer_llm,answer_orig,document,question,course
10,"Yes, all sessions are recorded, so if you miss...","Everything is recorded, so you won’t miss anyt...",5170565b,Are sessions recorded if I miss one?,machine-learning-zoomcamp
11,"Yes, you can ask your questions in advance if ...","Everything is recorded, so you won’t miss anyt...",5170565b,Can I ask questions in advance if I can't atte...,machine-learning-zoomcamp
12,"If you miss a session, don't worry! Everything...","Everything is recorded, so you won’t miss anyt...",5170565b,How will my questions be addressed if I miss a...,machine-learning-zoomcamp
13,"Yes, there is a way to catch up on a missed se...","Everything is recorded, so you won’t miss anyt...",5170565b,Is there a way to catch up on a missed session?,machine-learning-zoomcamp
14,"Yes, you can still interact with instructors a...","Everything is recorded, so you won’t miss anyt...",5170565b,Can I still interact with instructors after mi...,machine-learning-zoomcamp


In [38]:
df = df.iloc[:300]

In [5]:
model_name='multi-qa-mpnet-base-dot-v1'
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

/home/gary/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-28 15:31:43.517487: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-28 15:31:44.643738: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
answer_llm = df.iloc[0].answer_llm

In [7]:
embs = embedding_model.encode(answer_llm)

In [8]:
for i in embs:
    print("{:f}".format(i))
    break

-0.422447


In [9]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm =  record['answer_llm']

    v_orig = embedding_model.encode(answer_orig)
    v_llm = embedding_model.encode(answer_llm)

    return v_llm.dot(v_orig)

In [10]:
results_gpt4o = df.to_dict(orient='records')

In [11]:
from tqdm import tqdm
similarity = []

for record in tqdm(results_gpt4o):
    sim = compute_similarity(record)
    similarity.append(sim)

100%|██████████| 300/300 [00:06<00:00, 44.16it/s]


In [12]:
df['cosine'] = similarity

In [13]:
df['cosine'].describe()

count    300.000000
mean      27.495996
std        6.384745
min        4.547924
25%       24.307850
50%       28.336861
75%       31.674302
max       39.476017
Name: cosine, dtype: float64

In [14]:
import numpy as np

In [15]:
norm = np.sqrt(( df['cosine'] *  df['cosine']).sum())

In [16]:
norm

488.87387

In [26]:
v_norm = df['cosine'] / norm

In [28]:
v_norm.describe()

count    300.000000
mean       0.056244
std        0.013060
min        0.009303
25%        0.049722
50%        0.057964
75%        0.064790
max        0.080749
Name: cosine, dtype: float64

In [23]:
df.head(11)

,answer_llm,answer_orig,document,question,course,cosine
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,17.515991
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,13.418406
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,25.313248
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,12.147420
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,18.747726
5,"The course videos are pre-recorded, and you ca...","The course videos are pre-recorded, you can st...",39fda9f0,Are the course videos live or pre-recorded?,machine-learning-zoomcamp,33.970398
6,You can start watching the course videos right...,"The course videos are pre-recorded, you can st...",39fda9f0,When can I start watching the course videos?,machine-learning-zoomcamp,30.251701
7,"Yes, the live office hours sessions are recorded.","The course videos are pre-recorded, you can st...",39fda9f0,Are the live office hours sessions recorded?,machine-learning-zoomcamp,29.521587
8,You can find the office hours sessions in the ...,"The course videos are pre-recorded, you can st...",39fda9f0,Where can I find the office hours sessions?,machine-learning-zoomcamp,35.272209
9,You can access the pre-recorded course videos ...,"The course videos are pre-recorded, you can st...",39fda9f0,Where can I access the pre-recorded course vid...,machine-learning-zoomcamp,27.751755


In [27]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(df['answer_llm'][10], df['answer_orig'][10])[0]

In [26]:
df['answer_llm'][10]

"Yes, all sessions are recorded, so if you miss one, you won't miss anything. You can catch up on the content later. Additionally, you can submit your questions in advance for office hours, and those sessions are also recorded."

In [28]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [31]:
(scores['rouge-1']['f'] + scores['rouge-2']['f'] + scores['rouge-l']['f']) / 3

0.35490034990035496

In [32]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(df['answer_llm'], df['answer_orig'])[0]

In [33]:
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

In [34]:
rouge_avg

0.07288173149369313

In [35]:
scores

{'rouge-1': {'r': 0.061224489795918366,
  'p': 0.21428571428571427,
  'f': 0.09523809178130524},
 'rouge-2': {'r': 0.017543859649122806,
  'p': 0.07142857142857142,
  'f': 0.028169010918468917},
 'rouge-l': {'r': 0.061224489795918366,
  'p': 0.21428571428571427,
  'f': 0.09523809178130524}}